<a href="https://colab.research.google.com/github/manishiitg/ML_Experiments/blob/master/recruit/glove_recruit_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install Flask
!pip install flask_pymongo

In [0]:
from flask import Flask
from flask_pymongo import PyMongo
from bs4 import BeautifulSoup
from nltk.tokenize import ToktokTokenizer
from nltk import sent_tokenize, word_tokenize

In [0]:
#__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
# __label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
# __label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
# __label__restaurant Michelin Three Star Restaurant; but if the chef is not there
# sample format needed


app = Flask(__name__)
app.config["MONGO_URI"] = ""
mongo = PyMongo(app)


def cleanMe(html):
    # create a new bs4 object from the html data loaded
    
    # print(html)
    soup = BeautifulSoup(html, "html.parser")
    # remove all javascript and stylesheet code
    for script in soup(["script", "style"]):
        script.extract()
    # get text
    text = soup.get_text(separator=' ')

    # break into lines and remove leading and trailing space on each
    # lines = (line.strip() for line in text.splitlines())
    lines = text.splitlines()
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    # text = '\n'.join(chunk for chunk in chunks if chunk)
    chunks = filter(None, chunks)
    return " ".join(chunks)

ret = mongo.db.naukri_candidates.find({})

all_lines = []


for row in ret:
  education = row["education"]
  workExperiance = row["workExperiance"]
  summary = row['summary']
  key_skills = row["key_skills"]
  projects = row["projects"]
  # extra_data = row["extra_data"]

  summary_str = summary.splitlines()

  education_string = []
  for edu in education:
      education_string.append(edu["type"]  + " " + edu["degree"] + " " + edu["specific"] + " " + edu["university"] + " " + edu["year"])

  work_exp_string = []
  for work in workExperiance:
      work_exp_string.append(work["company_name"]  + " " + work["designation"] + " " + work["date"] + " " + cleanMe(work["desc"]))

  key_skills_str =  " ".join(key_skills.split(",")).replace("Key Skills:","")

  project_str = []
  for project in projects:
      project_str.append(cleanMe(project))

  all_lines.extend(summary_str)
  all_lines.extend(education_string)
  all_lines.extend(work_exp_string)
  all_lines.append(key_skills_str)
  all_lines.extend(project_str)

      # print(row)

print("total lines ", len(all_lines))
            

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


total lines  387340


In [0]:
import nltk
nltk.download('punkt')

toktok = ToktokTokenizer()


open('glove.train', 'w').close()


with open("glove.train", 'a') as the_file2:
  for line in all_lines:
    tokens = [toktok.tokenize(sent) for sent in sent_tokenize(line)]
    for sent in tokens:
      the_file2.write(" ".join(sent) + ' ')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
!head -c 20 glove.train

Offering 3.5 years o

In [0]:
!pip install glove_python
from glove import Corpus, Glove

def read_corpus(filename):

    delchars = [chr(c) for c in range(256)]
    delchars = [x for x in delchars if not x.isalnum()]
    delchars.remove(' ')
    delchars = ''.join(delchars)

    table = str.maketrans(dict.fromkeys(delchars))

    with open(filename, 'r') as datafile:
        for line in datafile:
            yield line.lower().translate(table).split(' ')

#Creating a corpus object
corpus = Corpus() 

#Training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(read_corpus("fastText.train"), window=10)

glove = Glove(no_components=5, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=100, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

In [0]:
glove = Glove.load('glove.model')

print('Querying for php')
print(glove.most_similar("node", number=10))

In [0]:
print(glove.most_similar("php", number=10))

In [0]:
print(glove.most_similar("reactjs", number=10))

In [0]:
!git clone http://github.com/stanfordnlp/glove

Cloning into 'glove'...
remote: Enumerating objects: 436, done.
remote: Total 436 (delta 0), reused 0 (delta 0), pack-reused 436
Receiving objects: 100% (436/436), 179.83 KiB | 6.92 MiB/s, done.
Resolving deltas: 100% (239/239), done.


In [0]:
!cd glove && make

mkdir -p build
gcc src/glove.c -o build/glove -lm -pthread -Ofast -march=native -funroll-loops -Wall -Wextra -Wpedantic
src/glove.c: In function ‘glove_thread’:
src/glove.c:117:9: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
         fread(&cr, sizeof(CREC), 1, fin);
         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
gcc src/shuffle.c -o build/shuffle -lm -pthread -Ofast -march=native -funroll-loops -Wall -Wextra -Wpedantic
src/shuffle.c: In function ‘shuffle_merge’:
src/shuffle.c:106:17: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
                 fread(&array[i], sizeof(CREC), 1, fid[j]);
                 ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/shuffle.c: In function ‘shuffle_by_chunks’:
src/shuffle.c:163:9: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
         fread(&array[i], sizeof(CREC), 1, fin);
         ^~

In [0]:
# don't run this code. copy it to a file demo_recruit.sh

#!/bin/bash
set -e

# Makes programs, downloads sample data, trains a GloVe model, and then evaluates it.
# One optional argument can specify the language used for eval script: matlab, octave or [default] python

make

CORPUS=../glove.train
VOCAB_FILE=vocab.txt
COOCCURRENCE_FILE=cooccurrence.bin
COOCCURRENCE_SHUF_FILE=cooccurrence.shuf.bin
BUILDDIR=build
SAVE_FILE=vectors
VERBOSE=2
MEMORY=4.0
VOCAB_MIN_COUNT=5
VECTOR_SIZE=300
MAX_ITER=15
WINDOW_SIZE=15
BINARY=2
NUM_THREADS=8
X_MAX=10

echo
echo "$ $BUILDDIR/vocab_count -min-count $VOCAB_MIN_COUNT -verbose $VERBOSE < $CORPUS > $VOCAB_FILE"
$BUILDDIR/vocab_count -min-count $VOCAB_MIN_COUNT -verbose $VERBOSE < $CORPUS > $VOCAB_FILE
echo "$ $BUILDDIR/cooccur -memory $MEMORY -vocab-file $VOCAB_FILE -verbose $VERBOSE -window-size $WINDOW_SIZE < $CORPUS > $COOCCURRENCE_FILE"
$BUILDDIR/cooccur -memory $MEMORY -vocab-file $VOCAB_FILE -verbose $VERBOSE -window-size $WINDOW_SIZE < $CORPUS > $COOCCURRENCE_FILE
echo "$ $BUILDDIR/shuffle -memory $MEMORY -verbose $VERBOSE < $COOCCURRENCE_FILE > $COOCCURRENCE_SHUF_FILE"
$BUILDDIR/shuffle -memory $MEMORY -verbose $VERBOSE < $COOCCURRENCE_FILE > $COOCCURRENCE_SHUF_FILE
echo "$ $BUILDDIR/glove -save-file $SAVE_FILE -threads $NUM_THREADS -input-file $COOCCURRENCE_SHUF_FILE -x-max $X_MAX -iter $MAX_ITER -vector-size $VECTOR_SIZE -binary $BINARY -vocab-file $VOCAB_FILE -verbose $VERBOSE"
$BUILDDIR/glove -save-file $SAVE_FILE -threads $NUM_THREADS -input-file $COOCCURRENCE_SHUF_FILE -x-max $X_MAX -iter $MAX_ITER -vector-size $VECTOR_SIZE -binary $BINARY -vocab-file $VOCAB_FILE -verbose $VERBOSE


In [0]:
!mv demo_recruit.sh glove
!chmod +x glove/demo_recruit.sh

In [0]:
!cd glove && ./demo_recruit.sh

mkdir -p build
gcc src/glove.c -o build/glove -lm -pthread -Ofast -march=native -funroll-loops -Wall -Wextra -Wpedantic
src/glove.c: In function ‘glove_thread’:
src/glove.c:117:9: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
         fread(&cr, sizeof(CREC), 1, fin);
         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
gcc src/shuffle.c -o build/shuffle -lm -pthread -Ofast -march=native -funroll-loops -Wall -Wextra -Wpedantic
src/shuffle.c: In function ‘shuffle_merge’:
src/shuffle.c:106:17: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
                 fread(&array[i], sizeof(CREC), 1, fid[j]);
                 ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/shuffle.c: In function ‘shuffle_by_chunks’:
src/shuffle.c:163:9: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
         fread(&array[i], sizeof(CREC), 1, fin);
         ^~

In [0]:
!pip install pymagnitude

     |████████████████████████████████| 5.4MB 2.6MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.120-cp36-cp36m-linux_x86_64.whl size=135918205 sha256=67d6a3468352335876ff505bd860c3237575b5b51f34003129187933a6a1b51c
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude


In [0]:
!python -m pymagnitude.converter -i glove/vectors.txt -o recruit_glove.magnitude

Detected GloVe format! Converting to word2vec format first...(this may take some time)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
Loading vectors... (this may take some time)
Found 66309 key(s)
Each vector has 300 dimension(s)
Creating magnitude format...
Writing vectors... (this may take some time)
0% completed
1% completed
2% completed
3% completed
4% completed
5% completed
6% completed
7% completed
8% completed
9% completed
10% completed
11% completed
12% completed
13% completed
14% completed
15% completed
16% completed
17% completed
18% completed
19% completed
20% completed
21% completed
22% completed
23% completed
24% completed
25% completed
26% completed
27% completed
28% co

In [0]:
from pymagnitude import *
vectors = Magnitude("recruit_glove.magnitude")

In [0]:
vectors.most_similar("smm", topn = 50) 

[('SM', 0.6379199408741184),
 ('SMM', 0.6195703464714559),
 ('SEM', 0.4732771027362554),
 ('SEO', 0.4189628086694664),
 ('SMO', 0.36017548277818057),
 ('PPC', 0.345457565060206),
 ('AdWords', 0.33357970975705487),
 ('Email', 0.32267457920517273),
 ('Website', 0.32250730239153397),
 ('Campaigns', 0.304301868989932),
 ('Branding', 0.3041796567207067),
 ('Campaign', 0.2988727501658046),
 ('Click', 0.2980303180898769),
 ('Facebook', 0.2917772049304623),
 ('Advertising', 0.28977580606001885),
 ('Media', 0.2872901278030426),
 ('Social', 0.28427356201594167),
 ('Offline', 0.2837374917028783),
 ('Promotions', 0.28095229089095003),
 ('Affiliate', 0.27478869372188436),
 ('Online', 0.2727894067095852),
 ('Marketing', 0.2698333216743065),
 ('Product/Brand', 0.26968630961979856),
 ('Instagram', 0.26119357895598044),
 ('Google', 0.257918933270143),
 ('Per', 0.25645881966039885),
 ('Adwords', 0.2552632086978399),
 ('CAP', 0.25288099591784685),
 ('ABM', 0.25162076199451644),
 ('101', 0.247409968411840

In [0]:
vectors.most_similar("consultant", topn = 10) 

[('Consultant', 0.6137663),
 ('analyst', 0.533376),
 ('associate', 0.5120199),
 ('consulting', 0.5083076),
 ('consultants', 0.5042535),
 ('architect', 0.4927193),
 ('erp', 0.4802565),
 ('sap', 0.47755232),
 ('engineer', 0.47225282),
 ('manager', 0.47155866)]

In [0]:
vectors.most_similar("infosys", topn = 20) 

[('Infosys', 0.8490976591422823),
 ('INFOSYS', 0.5975148372284107),
 ('Wipro', 0.5520590027970951),
 ('HCL', 0.5195024440467528),
 ('TCS', 0.510680427235336),
 ('Cognizant', 0.5037972894821914),
 ('Consultancy', 0.48767287626391553),
 ('Analyst', 0.4804347762588538),
 ('Associate', 0.47451889950275444),
 ('Accenture', 0.46954039206193743),
 ('Capgemini', 0.468406250714315),
 ('BPM', 0.46705215527415567),
 ('Technologies', 0.462012772911158),
 ('Senior', 0.45985927126168247),
 ('Consultant', 0.4452575155321057),
 ('TECHNOLOGIES', 0.44178186104580786),
 ('-IT', 0.43527168214017764),
 ('Engineer', 0.4315893119631141),
 ('Developer', 0.4223091144702823),
 ('BPO', 0.4222943348333804)]

In [0]:
vectors.most_similar(positive=["react"], negative=["php"], topn = 10) 

[('reproduce', 0.31450832),
 ('tackled', 0.31417537),
 ('transact', 0.3058074),
 ('absorb', 0.29586887),
 ('creatively', 0.29312435),
 ('Immediately', 0.28887305),
 ('Detected', 0.2838915),
 ('Adapts', 0.28123796),
 ('maternal', 0.2806481),
 ('EHP7', 0.28003758)]

In [0]:
vectors.most_similar("mca", topn = 50) 

[('MCA', 0.9195315182465855),
 ('Computers', 0.701173578411215),
 ('BCA', 0.6356204838437375),
 ('Maths', 0.5372515964734721),
 ('PG', 0.5095975731164931),
 ('B.Sc', 0.5053512167743351),
 ('Osmania', 0.4989245337839388),
 ('University', 0.4903417619697067),
 ('MS/M.Sc(', 0.48413652170094135),
 ('Electronics/Telecommunication', 0.4778023215539947),
 ('Gandhi', 0.47457254232262813),
 ('Indira', 0.47298987390718217),
 ('Kakatiya', 0.47269493970914683),
 ('Kamaraj', 0.4640060043109421),
 ('IGNOU', 0.4603008341823449),
 ('Madras', 0.45932181833306807),
 ('Science', 0.4568753958560985),
 ('Jawaharlal', 0.4459748567317503),
 ('Bharathidasan', 0.4454463849069672),
 ('Computer', 0.44308367118316505),
 ('2002', 0.4410391027446936),
 ('MBA/PGDM', 0.4387918365429997),
 ('SVU', 0.4377131114682043),
 ('JNTU', 0.43737481122859634),
 ('Venkateshwara', 0.4351001412961457),
 ('B515', 0.43474935920221314),
 ('MKU', 0.4328904262145933),
 ('M.Com', 0.4321484193000711),
 ('Utkal', 0.4285095158737191),
 ('19

In [0]:
vectors.most_similar(["technical","writing"], topn = 50) 

[('specifications', 0.611176),
 ('functional', 0.59229994),
 ('design', 0.5830507),
 ('requirements', 0.5806917),
 ('preparing', 0.57033736),
 ('documentation', 0.56332606),
 ('providing', 0.55973434),
 ('Involved', 0.5596165),
 ('Technical', 0.5572456),
 ('creating', 0.5518776),
 ('test', 0.55112636),
 ('gathering', 0.54319537),
 ('coding', 0.542947),
 ('requirement', 0.5416961),
 ('complex', 0.54087234),
 ('developing', 0.54025936),
 ('designing', 0.53916776),
 ('documents', 0.5374689),
 ('development', 0.53050196),
 ('support', 0.5279308),
 ('reviewing', 0.523856),
 ('detailed', 0.5196172),
 ('specification', 0.51706135),
 ('case', 0.5157835),
 ('document', 0.5111048),
 ('queries', 0.50520664),
 ('knowledge', 0.49727082),
 ('proposal', 0.49648023),
 ('cases', 0.49610817),
 ('Expertise', 0.4957046),
 ('Providing', 0.4951682),
 ('skills', 0.49436492),
 ('review', 0.49351817),
 ('Writing', 0.48898107),
 ('along', 0.48865792),
 ('including', 0.48520672),
 ('quality', 0.48441908),
 ('res

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -f *.magnitude /content/drive/My\ Drive/Colab\ Notebooks